In [26]:
# Import necessary dependencies
import pandas as pd
from sqlalchemy import create_engine, text
from sklearn import metrics
import pickle
import numpy as np
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import pymysql

from sklearn.linear_model import LinearRegression

In [27]:
URL = "dbbikes.cvwut6jnqsvn.us-east-1.rds.amazonaws.com"
PORT = "3306"
DB = "dbbikes"
USER = "admin"
PASSWORD = "password"

In [28]:
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{URL}:{PORT}/{DB}",echo = True)

# Weather Data Preparation
___

In [29]:
# create dataframe of weatherData
query = "SELECT * FROM dbbikes.Weather;"
dfWeather = pd.DataFrame(engine.connect().execute(text(query)))
dfWeather["last_update"] = pd.to_datetime(dfWeather.last_update) #change last_update feature to datetime type

2023-04-20 18:02:01,316 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2023-04-20 18:02:01,317 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-04-20 18:02:01,499 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2023-04-20 18:02:01,500 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-04-20 18:02:01,592 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2023-04-20 18:02:01,594 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-04-20 18:02:01,777 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-20 18:02:01,779 INFO sqlalchemy.engine.Engine SELECT * FROM dbbikes.Weather;
2023-04-20 18:02:01,784 INFO sqlalchemy.engine.Engine [generated in 0.00754s] {}


In [30]:
dfWeather.tail()

,coord_lon,coord_lat,weather_id,weather_main,weather_description,weather_icon,weather_base,main_temp,feels_like,main_temp_min,...,clouds_all,last_update,sys_type,sys_id,sys_country,sys_sunrise,sys_sunset,city_id,city_name,cod
4186,-6.2672,53.344,801,Clouds,few clouds,02d,stations,284.92,283.93,284.03,...,20,2023-04-20 17:35:10,2,2037117,IE,2023-04-20 06:14:57,2023-04-20 20:32:46,2964574,Dublin,200
4187,-6.2672,53.344,801,Clouds,few clouds,02d,stations,285.21,284.19,284.75,...,20,2023-04-20 17:45:00,2,2037117,IE,2023-04-20 06:14:57,2023-04-20 20:32:46,2964574,Dublin,200
4188,-6.2672,53.344,801,Clouds,few clouds,02d,stations,285.29,284.28,284.75,...,20,2023-04-20 17:47:40,2,2037117,IE,2023-04-20 06:14:57,2023-04-20 20:32:46,2964574,Dublin,200
4189,-6.2672,53.344,801,Clouds,few clouds,02d,stations,285.29,284.28,284.75,...,20,2023-04-20 17:50:03,2,2037117,IE,2023-04-20 06:14:57,2023-04-20 20:32:46,2964574,Dublin,200
4190,-6.2672,53.344,801,Clouds,few clouds,02d,stations,285.29,284.28,284.75,...,20,2023-04-20 17:56:13,2,2037117,IE,2023-04-20 06:14:57,2023-04-20 20:32:46,2964574,Dublin,200


In [31]:
# Select categorical columns
categorical_columns = dfWeather[['weather_main', 'weather_description', 'weather_icon', 'weather_base']].columns

# Convert data type to category for these columns
for column in categorical_columns:
    dfWeather[column] = dfWeather[column].astype('object')

In [32]:
# Select columns that should be floats
float_columns = dfWeather[['coord_lon', 'coord_lat', 'main_temp', 'feels_like',"main_temp_min","wind_speed",
                    "main_temp_max",]].columns
# Convert data type to floats
for column in float_columns:
    dfWeather[column] = dfWeather[column].astype('float64')

# Select data types that should be integers
int_columns=dfWeather[['weather_id', 'main_pressure', 'main_humidity', 'main_visibility',"clouds_all",]].columns

for column in int_columns:
    dfWeather[column] = dfWeather[column].astype('int64')


In [33]:
# Select only important Weather features
dfWeather = dfWeather[["weather_description","main_temp","main_humidity",'wind_speed',"last_update"]]

In [34]:
dfWeather.sort_values('last_update', inplace = True)

# Bikes Data Preparation
___

In [36]:
query = "SELECT * FROM dbbikes.Bike;"
dfBikes = pd.DataFrame(engine.connect().execute(text(query)))
dfBikes["last_update"] = pd.to_datetime(dfBikes.last_update)
# Select continuous values and change to appropriate type
dfBikes.sort_values('last_update', inplace = True)

2023-04-20 18:02:38,453 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-04-20 18:02:38,488 INFO sqlalchemy.engine.Engine SELECT * FROM dbbikes.Bike;
2023-04-20 18:02:38,491 INFO sqlalchemy.engine.Engine [generated in 0.03775s] {}


In [37]:
dfBikes.dtypes

number                            int64
contract_name                    object
name                             object
bike_stands                       int64
available_bike_stands             int64
available_bikes                   int64
status                           object
last_update              datetime64[ns]
dtype: object

#Merging the two Dataframes
___

In [38]:
dfMerged = pd.merge_asof(dfBikes,dfWeather,on="last_update",direction="nearest")

In [39]:
df = dfMerged.drop(["contract_name","name","status"],axis=1)

In [40]:
df['month'] = df['last_update'].dt.month
df['dayofweek'] = df['last_update'].dt.dayofweek
df['hour'] = df['last_update'].dt.hour
df['minute'] = df['last_update'].dt.minute

In [41]:
df.dtypes


number                            int64
bike_stands                       int64
available_bike_stands             int64
available_bikes                   int64
last_update              datetime64[ns]
weather_description              object
main_temp                       float64
main_humidity                     int64
wind_speed                      float64
month                             int64
dayofweek                         int64
hour                              int64
minute                            int64
dtype: object

In [42]:
df.tail(1000)


,number,bike_stands,available_bike_stands,available_bikes,last_update,weather_description,main_temp,main_humidity,wind_speed,month,dayofweek,hour,minute
312449,88,30,12,18,2023-04-20 17:05:12,few clouds,285.30,65,8.75,4,3,17,5
312450,30,20,13,7,2023-04-20 17:05:15,few clouds,285.30,65,8.75,4,3,17,5
312451,36,40,37,3,2023-04-20 17:05:17,few clouds,285.30,65,8.75,4,3,17,5
312452,42,30,14,16,2023-04-20 17:05:25,few clouds,285.30,65,8.75,4,3,17,5
312453,21,30,2,28,2023-04-20 17:05:25,few clouds,285.30,65,8.75,4,3,17,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
313444,7,29,29,0,2023-04-20 17:59:49,few clouds,285.29,66,7.72,4,3,17,59
313445,91,30,12,18,2023-04-20 17:59:49,few clouds,285.29,66,7.72,4,3,17,59
313446,86,38,0,38,2023-04-20 17:59:50,few clouds,285.29,66,7.72,4,3,17,59
313447,24,20,4,16,2023-04-20 17:59:50,few clouds,285.29,66,7.72,4,3,17,59


In [43]:
df.to_csv("mergedData.csv")
df.dtypes

number                            int64
bike_stands                       int64
available_bike_stands             int64
available_bikes                   int64
last_update              datetime64[ns]
weather_description              object
main_temp                       float64
main_humidity                     int64
wind_speed                      float64
month                             int64
dayofweek                         int64
hour                              int64
minute                            int64
dtype: object

# Training Data Model
___

In [44]:
# Transform weather description to integers
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
# Get the unique values in the 'weather_description' column
unique_values = df['weather_description'].unique()

df['weather_description'] = label_encoder.fit_transform(df['weather_description'])

In [45]:
# Get the corresponding numerical labels for the unique values
numerical_labels = label_encoder.transform(unique_values)

# Create a dictionary of features and their encoded numerical data
feature_dict = {}
for i in range(len(unique_values)):
    feature_dict[unique_values[i]] = numerical_labels[i]

In [46]:
feature_dict

{'broken clouds': 0,
 'light rain': 8,
 'scattered clouds': 13,
 'light intensity shower rain': 7,
 'light intensity drizzle': 5,
 'mist': 9,
 'light intensity drizzle rain': 6,
 'moderate rain': 10,
 'overcast clouds': 11,
 'clear sky': 1,
 'few clouds': 2,
 'heavy intensity rain': 4,
 'ragged shower rain': 12,
 'fog': 3}

In [47]:
df.dtypes


number                            int64
bike_stands                       int64
available_bike_stands             int64
available_bikes                   int64
last_update              datetime64[ns]
weather_description               int32
main_temp                       float64
main_humidity                     int64
wind_speed                      float64
month                             int64
dayofweek                         int64
hour                              int64
minute                            int64
dtype: object

In [ ]:
df = df.drop("last_update",1)

In [ ]:
input_features = ['number','hour','minute','month','weather_description','main_temp', 'main_humidity', 'wind_speed','dayofweek']

In [ ]:
input = df[input_features]
target = df[["available_bikes",'dayofweek']]

In [ ]:
X = input
Y = target.available_bikes
X_train, X_test,Y_train,Y_test = train_test_split(X, Y, test_size=0.3,random_state=0)
regressor = RandomForestRegressor(n_estimators=15)
regressor.fit(X_train, Y_train)

# Write to a pickle file
with open('bike_predictor.pkl', 'wb') as handle:
    pickle.dump(regressor, handle, pickle.HIGHEST_PROTOCOL)


y_pred = regressor.predict(X_test)
df = pd.DataFrame({'Actual': Y_test, 'Predicted': y_pred})

print('Test Set Predictions for Station: ')
print(df)
print()

#Accuracy score
print('Mean Absolute Error:', metrics.mean_absolute_error(Y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(Y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(Y_test, y_pred)))
print ("The accuracy score : ",metrics.r2_score(Y_test, y_pred))


# Testing PKL file
___

In [ ]:
row = [0]*9

In [ ]:
x_testing = pd.DataFrame([row],columns=input_features)


In [ ]:
inputData = {'number':12,'hour': 10,'minute':30,'month': 4, 'weather_description': 3, 'main_temp': 279.60, 'main_humidity': 1025, 'wind_speed': 6.6, 'dayofweek': 3}

In [ ]:
def dataframe(df, dict):
    df['number'][0]=dict['number']
    df['hour'][0] = dict['hour']
    df['minute'][0] = dict['minute']
    df['month'][0] = dict['month']
    df['weather_description'][0] = dict['weather_description']
    df['main_temp'][0] = dict['main_temp']
    df['main_humidity'][0] = dict['main_humidity']
    df['wind_speed'][0] = dict['wind_speed']
    df['dayofweek'][0] = dict['dayofweek']

In [ ]:
dataframe(X,inputData)

In [ ]:
with open('bike_predictor.pkl', 'rb') as handle:
    regressor = pickle.load(handle)

prediction = regressor.predict(X)
prediction[0]
